Hello everyone!

## Winning Imports

Finding out how to import or what imports to use was extremely difficult. Mainly, because we needed a Data Collator that does a very niche thing that is not built in. So we needed to use TRL's DataCollatorForCompletionOnlyLM. I will try to explain this in more detail later. But for now I can share the import code that worked well. 

```
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
!pip install --upgrade -qqq uv
try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
except: get_numpy = "numpy"
try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
except: is_t4 = False
get_vllm, get_triton = ("vllm==0.10.1", "triton==3.2.0") if is_t4 else ("vllm", "triton")
!uv pip install -qqq --upgrade     unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers
!uv pip install -qqq {get_triton}
!uv pip install "huggingface_hub>=0.34.0" "datasets>=3.4.1,<4.0.
!uv pip install transformers==4.55.4
!uv pip install "trl==0.9.6"
```

Now you might be wondering what all of this does. 

## Data Creation

Data creation is an interesting case. Because we need to do several things. We first need to process each task into a form that we intuitively think LLMs will have an easier time understanding. 

The original grind world problems are represented by JSON. So we have a bunch of unique task keys.

So for each task the task data looks like this:
```
{test: {input: gridArray} ,
 train: [
    {input: gridArray,
     output: gridArray},
    {input: gridArray,
     output: gridArray}, ...
 ]}
```
and then the solution data looks like this:
```
[gridArray]
```
Note: sometimes there are multiple test inputs and solution outputs

Now, we need to transform this into something that the ML model will have a less messy, easier time reading and have the proper format for our data loaders in training to be able to us. 

Note: we know what we want it to look like so now we just have to figure out what our data loader would expect.

# Data Collator

This was a very difficult part of the pr

Okay what am I trying to do.
I want to get the fine-tuning pipeline build using multiple gpus.
I want to use somewhat basic configs so they are simple for me to understand.
I want to have a clear file system that is easy to navigate
The end goal is to have a fine-tuned model sent to my huggingface folder
I think I will have to merge the weights together before sending them to huggingface
I need the data to be created and then sent through the pipeline. I want to do DDP.
I should figure out if they create the data and then send put it in the data collator or what
So I need to set up data infra training infra and merging and sending infra



/shared/arc/
├── data/                       # read-only datasets (everyone can read)
│   └── ARC-Data/
│       └── input/
│           ├── arc-prize-2024/
│           │   ├── arc-agi_evaluation_challenges.json
│           │   └── arc-agi_evaluation_solutions.json
│           ├── re_arc/
│           │   ├── metadata.json
│           │   └── tasks/
│           │       ├── <task-id-1>.json
│           │       └── ...
│           └── arc-dataset-collection/
│               └── dataset/
│                   └── ConceptARC/
│                       └── data/
│                           ├── group-1/*.json
│                           └── ...
├── outputs/                    # shared, writable (checkpoints, logs, merged models)
│   ├── runs/                   # per-run folders
│   ├── checkpoints/
│   ├── logs/
│   └── models/
│       └── Llama-3/            # where your script saves LoRA + merged
└── cache/
    ├── hf/                     # shared HF cache (optional)
    └── ds/                     # optional DeepSpeed NVMe/AIO cache (if used)

main/
├── code/
│   └── arc-trainer/            # your git repo (your Python package + scripts)
│       ├── train_v1.py
│       ├── ds_configs/
│       │   ├── ds_ze[email protected]          # deepspeed jsons
│       │   └── ds_zero2.json
│       ├── configs/
│       │   └── training.yaml   # hyperparams you may want in YAML
│       ├── scripts/
│       │   ├── run_slurm.sh    # SLURM launcher
│       │   └── run_local.sh    # single-node launcher
│       └── README.md
└── .cache/
    └── huggingface/            # per-user HF cache (if not using shared)

/scratch/$USER/                 # fast temp (per-job)
└── arc-run-<jobid>/            # created by run script at runtime
    ├── offload/                # DS CPU/NVMe offload (if enabled)
    └── tmp/                    # temp files
